# MultiHop Planner Agent

### Load the Data and create Tools

In [ ]:
# !mkdir -p 'data/10k/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(
    model="gpt-4o",
    temperature=0.1,
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool

# Load documents, create tools
lyft_documents = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_documents = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

lyft_index = VectorStoreIndex.from_documents(lyft_documents)
uber_index = VectorStoreIndex.from_documents(uber_documents)

lyft_tool = QueryEngineTool.from_defaults(
    lyft_index.as_query_engine(),
    name="lyft_2021",
    description="Useful for asking questions about Lyft's 2021 10-K filling.",
)

uber_tool = QueryEngineTool.from_defaults(
    uber_index.as_query_engine(),
    name="uber_2021",
    description="Useful for asking questions about Uber's 2021 10-K filling.",
)

### Create Agent

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.agent.multi_hop.planner import MultiHopPlannerAgent

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools([lyft_tool, uber_tool], verbose=True)

# wrap the worker in the top-level planner
agent = MultiHopPlannerAgent(worker, tools=[lyft_tool, uber_tool], verbose=True)

In [ ]:
agent.create_plan(input="What was Lyft's revenue growth in 2021?")

=== Initial plan ===
## Structured Context
{
    "title": "StructuredContext",
    "description": "Data class for holding data requirements to answer query",
    "type": "object",
    "properties": {
        "lyfts_revenue_in_2020": {
            "title": "Lyfts Revenue In 2020",
            "description": "The total revenue generated by Lyft in the year 2020.",
            "type": "string"
        },
        "lyfts_revenue_in_2021": {
            "title": "Lyfts Revenue In 2021",
            "description": "The total revenue generated by Lyft in the year 2021.",
            "type": "string"
        },
        "lyfts_revenue_growth_in_2021": {
            "title": "Lyfts Revenue Growth In 2021",
            "description": "The percentage or amount of revenue growth experienced by Lyft in the year 2021.",
            "type": "string"
        }
    }
}

## Sub Tasks
lyfts_revenue_in_2020:
Extract this data field. -> The total revenue generated by Lyft in the year 2020.
deps: []


lyfts_r

'a9ab9933-a18c-4038-a863-6061339c8251'